# Data Product Arquitecture - Food Inspections Project

Team:

* Jóse Roberto Pérez - 173013
* Santiago Battezzati - 197727
* Carlos Geovany Pérez - 197614
* Mariana Lugo - 118626

## Imports

#### Standard library imports

In [1]:
import os
import sys
from datetime import datetime

#### Third party imports

In [2]:
import pandas as pd
from pandas_profiling import ProfileReport
pd.options.display.max_columns = 25

import seaborn as sns

#### Local application imports

In [3]:
sys.path.append("../")

In [21]:
%reload_ext autoreload
%autoreload 2

from src.utils.utils import *

---

## Importing and cleaning data
Food Inspections in Chicago: https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5

In [36]:
df = pd.read_csv('../data/Food_Inspections.csv')
df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2453552,CLAMP DOWN BURGERS,NaN,2749943.0,NaN,Risk 2 (Medium),1742 W DIVISION ST,CHICAGO,IL,60622.0,10/20/2020,License,No Entry,NaN,41.903387,-87.671740,"(-87.67174026586648, 41.903386755553484)"
1,2386633,JIN JU,JIN JU,27137.0,Restaurant,Risk 1 (High),5203 N CLARK ST,CHICAGO,IL,60640.0,08/28/2020,Canvass,No Entry,NaN,41.976301,-87.668276,"(-87.66827593789948, 41.97630115368914)"
2,2386595,LA BIZNAGA #2,LA BIZNAGA #2,2708992.0,NaN,Risk 1 (High),2949 W BELMONT AVE,CHICAGO,IL,60618.0,08/27/2020,Complaint,No Entry,NaN,41.939256,-87.702270,"(-87.70226967930802, 41.939255926667535)"
3,2386464,Uni Sushi Bristo,Uni Sushi Bistro,2262637.0,Restaurant,Risk 1 (High),1752 W NORTH AVE,CHICAGO,IL,60622.0,08/25/2020,Complaint,No Entry,NaN,41.910676,-87.672205,"(-87.67220465807979, 41.91067561170382)"
4,2386398,KIKI'S BISTRO,KIKI'S BISTRO,22899.0,Restaurant,Risk 1 (High),900 N FRANKLIN ST,CHICAGO,IL,60610.0,08/24/2020,Canvass,No Entry,NaN,41.898998,-87.635921,"(-87.63592067312285, 41.89899799424835)"


Limpieza de nombres de las columnas:

In [37]:
clean_col_names(df)

,inspection_id,dba_name,aka_name,license_#,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2453552,CLAMP DOWN BURGERS,NaN,2749943.0,NaN,Risk 2 (Medium),1742 W DIVISION ST,CHICAGO,IL,60622.0,10/20/2020,License,No Entry,NaN,41.903387,-87.671740,"(-87.67174026586648, 41.903386755553484)"
1,2386633,JIN JU,JIN JU,27137.0,Restaurant,Risk 1 (High),5203 N CLARK ST,CHICAGO,IL,60640.0,08/28/2020,Canvass,No Entry,NaN,41.976301,-87.668276,"(-87.66827593789948, 41.97630115368914)"
2,2386595,LA BIZNAGA #2,LA BIZNAGA #2,2708992.0,NaN,Risk 1 (High),2949 W BELMONT AVE,CHICAGO,IL,60618.0,08/27/2020,Complaint,No Entry,NaN,41.939256,-87.702270,"(-87.70226967930802, 41.939255926667535)"
3,2386464,Uni Sushi Bristo,Uni Sushi Bistro,2262637.0,Restaurant,Risk 1 (High),1752 W NORTH AVE,CHICAGO,IL,60622.0,08/25/2020,Complaint,No Entry,NaN,41.910676,-87.672205,"(-87.67220465807979, 41.91067561170382)"
4,2386398,KIKI'S BISTRO,KIKI'S BISTRO,22899.0,Restaurant,Risk 1 (High),900 N FRANKLIN ST,CHICAGO,IL,60610.0,08/24/2020,Canvass,No Entry,NaN,41.898998,-87.635921,"(-87.63592067312285, 41.89899799424835)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215062,229233,"ZULLO'S MARKETS, LLC",ZULLO'S MARKET,2016915.0,Restaurant,Risk 2 (Medium),131 N Clinton ST,CHICAGO,IL,60661.0,02/18/2010,License,Fail,NaN,41.884188,-87.641120,"(-87.64111966683218, 41.884187507127805)"
215063,68205,LA FONDA CHIQUITA TLC,LA FONDA CHIQUITA,2017215.0,Restaurant,Risk 1 (High),5940 W DIVERSEY AVE,CHICAGO,IL,60639.0,02/09/2010,License,Fail,12. HAND WASHING FACILITIES: WITH SOAP AND SAN...,41.931260,-87.775203,"(-87.77520287598688, 41.93125971874477)"
215064,88234,KABAB CUISINE 2,KABAB CUISINE 2,2014076.0,Restaurant,Risk 1 (High),6320 N LINCOLN AVE,CHICAGO,IL,60659.0,02/01/2010,License Re-Inspection,Pass,NaN,41.996139,-87.716968,"(-87.71696834498202, 41.996139331170895)"
215065,74311,MARGARITA DISTRIBUTORS,NaN,2017298.0,Wholesale,Risk 3 (Low),2332 S BLUE ISLAND AVE BLDG,CHICAGO,IL,60608.0,02/19/2010,License Re-Inspection,Pass,NaN,41.849940,-87.672421,"(-87.67242100722251, 41.84994008002549)"


**Número de variables:**

In [38]:
count_vars(df)

Número de variables en los datos --> 17


**Número de observaciones:**

In [39]:
count_obs(df)

Número de observaciones en los datos --> 215067


**¿Qué variables son?**

In [40]:
count_unique_obs(df)

inspection_id      215067
dba_name            28748
aka_name            27355
license_#           39103
facility_type         500
risk                    4
address             18522
city                   70
state                   4
zip                   112
inspection_date      2796
inspection_type       110
results                 7
violations         156694
latitude            17246
longitude           17246
location            17246
dtype: int64

In [41]:
cat_vars=['dba_name','aka_name','facility_type','risk','address','city','state','inspection_type',
          'results','violations']
data_profiling_categ(df, cat_vars)

*********************************
Variable Categorica dba_name
*********************************


Info,dba_name
Num_Registros,215067
Num_de_categorias,28748
Moda,SUBWAY
Valores_faltantes,0
Top1,"[SUBWAY, 2905]"
Top2,"[DUNKIN DONUTS, 1578]"
Top3,"[MCDONALD'S, 614]"


None
Valores de las categorias y sus proporciones


,Observaciones,proporción
Categoría,,
SUBWAY,2905,1.4%
DUNKIN DONUTS,1578,0.7%
MCDONALD'S,614,0.3%
7-ELEVEN,469,0.2%
CHIPOTLE MEXICAN GRILL,344,0.2%
...,...,...
SUGAR,1,0.0%
BIRDIE PAK PRODUCTS INC,1,0.0%
KING FISH & WING,1,0.0%


None



*********************************
Variable Categorica aka_name
*********************************


...

None



*********************************
Variable Categorica violations
*********************************


Info,violations
Num_Registros,215067
Num_de_categorias,156694
Moda,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...
Valores_faltantes,57424
Top1,[32. FOOD AND NON-FOOD CONTACT SURFACES PROPER...
Top2,"[30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABE..."
Top3,[45. FOOD HANDLER REQUIREMENTS MET - Comments:...


None
Valores de las categorias y sus proporciones


,Observaciones,proporción
Categoría,,
"32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: VIOLATION STILL PENDING",11,0.0%
"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABELED: CUSTOMER ADVISORY POSTED AS NEEDED - Comments: OBSERVED NO DBA NAME ON THE LATERAL SIDE OF THE CART, INSTRUCTED TO PROVIDE BUSINESS NAME LEGIBLY PAINTED IN LETTERS AT LEAST TWO INCHES IN HEIGHT IN A CONSPICUOUS PLACE ON EACH LATERAL SIDE OF CART.",10,0.0%
45. FOOD HANDLER REQUIREMENTS MET - Comments: ALL FOOD HANDLING EMPLOYEES MUST SHOW PROOF OF FOOD HANDLER TRAINING. MUST PROVIDE.,10,0.0%
"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING - Comments: 2-102.14(N) FOUND NO EMPLOYEE HEALTH POLICY AT PREMISES. PRIORITY FOUNDATION. NO CITATION ISSUED. 7-38-010. MUST PROVIDE. | 5. PROCEDURES FOR RESPONDING TO VOMITING AND DIARRHEAL EVENTS - Comments: 2-501.11 FOUND NO VOMIT AND DIARRHEA CLEAN UP POLICY. PRIORITY FOUNDATION. NO CITATION ISSUED.7-38-005. MUST PROVIDE.",7,0.0%
45. FOOD HANDLER REQUIREMENTS MET - Comments: ALL FOOD HANDLING EMPLOYEES MUST SHOW PROOF OF FOOD HANDLER TRAINING. MUST PROVIDE.,7,0.0%
...,...,...
"5. PROCEDURES FOR RESPONDING TO VOMITING AND DIARRHEAL EVENTS - Comments: OBSERVED INCOMPLETE KIT FOR THE CLEAN UP OF VOMIT AND DIARRHEAL EVENTS. MUST PROVIDE A SANITIZER THAT IS EFFECTIVE AGAINST NOROVIRUS. PRIORITY FOUNDATION VIOLATION 7-38-005. NO CITATION ISSUED.\t | 49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Comments: OBSERVED EXCESSIVE GREASE AND DUST ON CORDS BEHIND OVENS AND ON BACK OF 1 DOOR UPRIGHT FREEZER. OBSERVED EXCESSIVE DEBRIS ON BAKERY SHELVES IN PREP AREA. OBSERVED EXCESSIVE BUILD UP ON SHELVES INSIDE WALK IN COOLER. MUST DETAIL CLEAN AND SANITIZE ALL EQUIPMENT. MUST MAINTAIN SAME. | 55. PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN - Comments: OBSERVED EXCESSIVE DEBRIS (COFFEE GRINDS, DUST, DIRT, FOOD, ETC.) ON FLOORS ALONG WALLS, BEHIND EQUIPMENT THROUGHOUT PREP AREA. MUST PULL ALL EQUIPMENT AWAY FROM WALLS TO DETAIL CLEAN FLOORS. OBSERVED LIQUID ON FLOOR OF WALK IN COOLER. MUST CLEAN AND SANITIZE. MUST KEEP CLEAN AND DRY. MUST MAINTAIN SAME.\t",1,0.0%
57. ALL FOOD EMPLOYEES HAVE FOOD HANDLER TRAINING - Comments: FOUND NO PROOF OFFOOD HANDLER TRAINING ON SITE. MUST PROOF OF FOOD HANDLER TRAINING FOR ALL FOOD HANDLERS.,1,0.0%
"18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: FOUND THE REAR EXIT DOOR LEADING TO THE OUTSIDE BETWEEN THE STORAGE AREA AND RESTAURANT NOT RODENT PROOFED AS REQUIRED. A 1/4 INCH GAP WAS OBSERVED AS THE DOOR SWEEP DID NOT COMPLETELY SEAL THE BOTTOM OF THE DOOR; ADVISED TO INSTALL A THRESH HOLE TO COMPLETELY SEAL BOTTOM OF DOOR. ALSO OBSESRVED TWENTY MOUSE DROPPINGS ON THE FLOOR UNDER THE STAIRS WHERE THIS DOOR IS LOCATED THAT IS NOT RODENT PROOFED; EXCESSIVE CLUTTER FOUND HERE; MUST CLEAN AND REMOVE DROPPINGS AND EXCESSIVE CLUTTER. SERIOUS VIOLATION 7-38-020 ISSUED. | 21. * CERTIFIED FOOD MANAGER ON SITE WHEN POTENTIALLY HAZARDOUS FOODS ARE PREPARED AND SERVED - Comments: NO CITY OF CHICAGO FOOD SERVICE MANAGER ON DUTY AS REQUIRED WHILE POTENTIALLY HAZARDOUS FOODS (SHRIMP, SEAFOOD, FISH) ARE BEING PREPARED AND SERVED; MUST HAVE A CITY OF CHICAGO CERTIFIED MANAGER ON DUTY AT ALL TIMES. SERIOUS VIOLATION 7-38-012 ISSUED. | 29. PREVIOUS MINOR VIOLATION(S) CORRECTED 7-42-090 - Comments: PREVIOUS MINOR VIOLATIONS NOT CORRECTED FROM JULY 17, 2014 AS DESCRIBED: #32 RUSTY SURFACES ON INTERIOR BOTTOM OF UPRIGHT COOLER; REMOVE SAME. #36 LIGHT OUT IN MAIN PREP AREA; REPLACE SAME. #40 METAL STEM THERMOMETER MISSING FOR FOOD HANDLER; PROVIDE SAME. #41 EXCESSIVE CLUTTER IN BASEMENT; INSTRUCTED TO REMOVE ANY UNNECESSARY ARTICLES AND ORGANIZE AREA. SERIOUS VIOLATION 7-42-090 ISSUED. | 32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: BUMPED UP TO A SERIOUS. | 32. FOOD AND NON

None





### Análisis de texto
Issue #13 GitHub

#### 1. Limpieza de campo `facility_type`

In [43]:
print(len(df["facility_type"].unique()))
df["facility_type"].unique()

501


array([nan, 'Restaurant', "Children's Services Facility", 'Grocery Store',
       'GAS STATION', 'Bakery', 'Long Term Care', 'Mobile Food Preparer',
       'School', 'Golden Diner', 'HOSTEL', 'school cafeteria',
       'Daycare (2 - 6 Years)', 'Daycare Above and Under 2 Years',
       'Hospital', 'Pop-Up Establishment Host-Tier II', 'Catering',
       'BREWERY', 'Liquor', 'TAVERN', 'Daycare Combo 1586',
       'Mobile Food Dispenser', 'SCHOOL', 'VENDING COMMISSARY',
       'Shared Kitchen User (Long Term)', 'COFFEE SHOP', 'Shared Kitchen',
       'LIQUOR/COFFEE KIOSK', 'SUPPORTIVE LIVING',
       'Daycare (Under 2 Years)', 'Mobile Prepared Food Vendor',
       'meat packing', 'CHARTER SCHOOL', 'PRIVATE SCHOOL',
       'Mobile Frozen Desserts Vendor', 'GROCERY STORE/COOKING SCHOOL',
       'CHURCH (SPECIAL EVENTS)', 'Special Event', 'STADIUM', 'Wholesale',
       'RESTAURANT/GROCERY STORE', 'bar', 'DISTRIBUTION CENTER',
       'Wrigley Roof Top', 'DAYCARE', 'PREPACAKAGED FOODS',
       

In [56]:
df["facility_type"].fillna("nan", inplace=True)

In [57]:
df["facility_type"] = df["facility_type"].apply(lambda x: clean_txt(x))

In [60]:
print(len(df["facility_type"].unique()))
df["facility_type"].value_counts()

448


restaurant                      143207
grocery_store                    28088
school                           13219
nan                               4889
children-s_services_facility      3707
                                 ...  
grocery-liquor                       1
religious                            1
mobil_food_1315                      1
kids_cafe-                           1
health_club                          1
Name: facility_type, Length: 448, dtype: int64

In [63]:
dfx = df["facility_type"].value_counts().to_frame()
dfx

,facility_type
restaurant,143207
grocery_store,28088
school,13219
nan,4889
children-s_services_facility,3707
...,...
grocery-liquor,1
religious,1
mobil_food_1315,1
kids_cafe-,1


In [69]:
dfx["facility_type"].value_counts()

2        66
1        63
3        59
4        30
5        26
         ..
57        1
63        1
65        1
66        1
28088     1
Name: facility_type, Length: 78, dtype: int64

In [78]:
mr1 = dfx["facility_type"] <= 1
mr2 = dfx.index.str.contains("rest")
dfx.loc[(mr2), :][:50]

,facility_type
restaurant,143207
grocery-restaurant,57
restaurant-bar,41
grocery_-_restaurant,29
restaurant-grocery_store,27
gas_station-restaurant,15
grocery-_restaurant,13
restaurant-hospital,11
restaurant.banquet_halls,9
rest-grocery,9


In [59]:
mr1 = df["facility_type"].str.contains("day")
df.loc[mr1, ["facility_type"]]

,facility_type
146,daycare_-2_-_6_years-
148,daycare_above_and_under_2_years
156,daycare_-2_-_6_years-
217,daycare_-2_-_6_years-
240,daycare_above_and_under_2_years
...,...
214998,daycare_-2_-_6_years-
214999,daycare_-2_-_6_years-
215023,daycare_-2_-_6_years-
215045,daycare_-2_-_6_years-


**Notas de análisis:**
- @9/Feb/2020
    - Una vez limpiados los textos, la cantidad de únicos pasó de 501 a 448.
    - Tal vez podríamos agrupar las categorías por aparición de palabras clave (e.g. `daycare` OR `day_care`).
    - Todas las que caigan fuera de las categorías, serán incluidas en una reserva llamada "Otros"
    - Podría valer la pena hacer un análisis de clústering para agrupar palabras por similitudes y así definir las categorías. (tal vez va a ser difícil hacer estas asociaciones.)

#### 2. Limpieza de campo `inspection_date`

#### 3. Limpieza de campo `violations`

---

## 1. ¿Cuáles son los tipos 10 de establecimientos que más violaciones cometen? (Facility Type)


In [ ]:
conteos_ft=df.groupby(["results","facility_type"]).size().reset_index(name="counts")
conteos_ft=conteos_ft.pivot("facility_type","results","counts")
conteos_ft=conteos_ft.fillna(0)
conteos_ft['Total']=conteos_ft.sum(1)
conteos_ft =conteos_ft.sort_values('Fail',ascending=False)
conteos_ft

In [ ]:
n = 10
top10=df['facility_type'].value_counts()[:10].sort_values(ascending=False)
top10

Se muestra el porcentaje del tipo de establecimiento acuerdo con el resultado de la inspección:

In [ ]:
proporciones_ft=round(conteos_ft.iloc[:,0:7].apply(lambda x: x/x.sum()*100, axis=1),2)
proporciones_ft=proporciones_ft.sort_values('Fail',ascending=False)
proporciones_ft

## 2. ¿Qué tipo de inspección detecta más establecimientos fuera de regla? (Inspection Type)

In [ ]:
conteos_it=df.groupby(["results","inspection_type"]).size().reset_index(name="counts")
conteos_it=conteos_it.pivot("inspection_type","results","counts")
conteos_it=conteos_it.fillna(0)
conteos_it['Total']=conteos_it.sum(1)
conteos_it =conteos_it.sort_values('Fail',ascending=False)
conteos_it

Se muestra el porcentaje del tipo de inspección de acuerdo con los resultados:

In [ ]:
proporciones_it=round(conteos_it.iloc[:,0:7].apply(lambda x: x/x.sum()*100, axis=1),2)
proporciones_it=proporciones_it.sort_values('Fail',ascending=False)
proporciones_it

Se muestra el top 20 de tipos de inspección con un porcentaje mayor de inspecciones no reprobadas.

In [ ]:
top20=proporciones_it.iloc[0:19,:]
top20

## 3. Relación entre el riesgo y resultado, ¿los más riesgosos no pasan la insepección?

In [ ]:
rr=df.groupby(['results','risk']).size().reset_index(name="counts")
rr=rr.pivot("results", "risk","counts")
rr=rr.fillna(0)
rr['Total']=rr.sum(1)
rr =rr.sort_values('Risk 1 (High)',ascending=False)
rr

In [ ]:
prop=round(rr.iloc[:,0:4].apply(lambda x: x/x.sum()*100, axis=1),2)
prop

In [ ]:
prop.plot(kind='bar',stacked=True)

## 4. ¿Cómo es el comportamiento de las inspecciones en el tiempo?

Se separa inspection_date en tres diferentes columnas: day, month, year y las agrega al dataframe:

In [ ]:
fecha= df.inspection_date.str.split("/", n=2,expand=True)
df['day']=fecha[1]
df['month']=fecha[0]
df['year']=fecha[2]

In [ ]:
df['inspection_date'] = pd.to_datetime(df['inspection_date'])

In [ ]:
# Columnas de year, month y counts
fecha_separated = df.groupby(['year','month']).size().reset_index(name="conteos")
# Hacemos tabla donde las columnas son años y las filas mes 
fecha_separated_p = fecha_separated.pivot("month", "year","conteos")
# Graficamos 
sns.set(style='whitegrid',rc={'figure.figsize':(10,10)},palette='Set3')
h =sns.lineplot(data=fecha_separated_p,linewidth = 3,dashes=False)
plt.legend(fontsize='x-large', title_fontsize='40',loc='lower right')
h.set(xlabel='Meses',ylabel='Conteos de Inspecciones')
plt.show()

## 5. ¿Cómo es el comportamiento por tipo de resultados en el tiempo?

In [ ]:
fecha_separated2 = df.groupby(['year','month','results']).size().reset_index(name="counts")
fecha_separated2['date']= fecha_separated2['year']+'-'+fecha_separated2['month']
g=fecha_separated2.drop(['year','month'],1)
g=g.set_index('date')
sns.set(style='whitegrid',rc={'figure.figsize':(20,10)},palette='Set2')
sns.lineplot(x='date',y='counts',hue='results',data=g,linewidth = 2)
plt.xticks(rotation= 90,size=8)
plt.show()

Se muestra el comportamiento anterior en porcentaje de inspecciones:

In [ ]:
g2=fecha_separated2.drop(['year','month'],1)
g2=g2.pivot("date", "results","counts")
g2=g2.fillna(0)
g2['total']=g2.sum(1)
prop2=round(g2.iloc[:,0:5].apply(lambda x: x/x.sum()*100, axis=1),2)
sns.set(style='whitegrid',rc={'figure.figsize':(20,10)},palette='Set2')
h =sns.lineplot(data=prop2,linewidth = 2,dashes=False,marker='o')
plt.xticks(rotation= 90)
plt.legend(fontsize='x-large', title_fontsize='40')
plt.show()

## 6. ¿Es importante la temporalidad de año, mes y día para las inspecciones?

Con variable inspection_date en formato tiempo creamos conteos del número de inspecciones hechas por día:

In [ ]:
conteos_fechas = df.groupby(['inspection_date']).size().reset_index(name='conteos')
# hacemos que inspection_date sea el identificador de las filas
conteos_fechas = conteos_fechas.set_index('inspection_date')


In [ ]:
# Separamos los días, meses, años
conteos_fechas.insert(1, 'year', conteos_fechas.index.year)
conteos_fechas.insert(2, 'month', conteos_fechas.index.month)
conteos_fechas.insert(3, 'day', conteos_fechas.index.day)
conteos_fechas = conteos_fechas.copy()

In [ ]:
# Graficamos
fig, axs = plt.subplots(figsize=(30,20),ncols=6, nrows=2)
heatmaps(conteos_fechas,2010,axs[0][0],'no','Day')
heatmaps(conteos_fechas,2011,axs[0][1],'no','')
heatmaps(conteos_fechas,2012,axs[0][2],'no','')
heatmaps(conteos_fechas,2013,axs[0][3],'no','')
heatmaps(conteos_fechas,2014,axs[0][4],'no','')
heatmaps(conteos_fechas,2015,axs[0][5],'yes','')
heatmaps(conteos_fechas,2016,axs[1][0],'no','')
heatmaps(conteos_fechas,2017,axs[1][1],'no','')
heatmaps(conteos_fechas,2018,axs[1][2],'no','')
heatmaps(conteos_fechas,2019,axs[1][3],'no','')
heatmaps(conteos_fechas,2020,axs[1][4],'no','')
heatmaps(conteos_fechas,2021,axs[1][5],'yes','')

**Vemos si para si existe un patrón temporal en el universo de inspecciones que resultaron como reprobadas (Results="Fail"):**

In [ ]:
fail=df[df.results=="Fail"]
len(fail)

In [ ]:
conteos_f = fail.groupby(['inspection_date']).size().reset_index(name='conteos')
conteos_f= conteos_f.set_index('inspection_date')
conteos_f.insert(1, 'year', conteos_f.index.year)
conteos_f.insert(2, 'month', conteos_f.index.month)
conteos_f.insert(3, 'day', conteos_f.index.day)
conteos_f = conteos_f.copy()

In [ ]:
fig, axs = plt.subplots(figsize=(30,20),ncols=6, nrows=2)
heatmaps(conteos_f,2010,axs[0][0],'no','Day')
heatmaps(conteos_f,2011,axs[0][1],'no','')
heatmaps(conteos_f,2012,axs[0][2],'no','')
heatmaps(conteos_f,2013,axs[0][3],'no','')
heatmaps(conteos_f,2014,axs[0][4],'no','')
heatmaps(conteos_f,2015,axs[0][5],'yes','')
heatmaps(conteos_f,2016,axs[1][0],'no','Day')
heatmaps(conteos_f,2017,axs[1][1],'no','')
heatmaps(conteos_f,2018,axs[1][2],'no','')
heatmaps(conteos_f,2019,axs[1][3],'no','')
heatmaps(conteos_f,2020,axs[1][4],'yes','')
heatmaps(conteos_f,2021,axs[1][5],'yes','')

## 7. Mapa

In [ ]:
sns.set(style='whitegrid',rc={'figure.figsize':(20,10)},palette='Set1')
s = sns.lmplot(x='longitude',y='latitude',data=df,hue='results',col='results',fit_reg=False,height=6,
               scatter_kws={"s":1,"alpha":.5})

In [ ]:
cols = ['Results','Latitude','Longitude']
rest = pd.read_csv('../data/Food_Inspections.csv', usecols=cols)

Eliminamos los registros sin coordenadas completas:

In [ ]:
rest = rest.dropna(subset=['Latitude'])
rest = rest.dropna(subset=['Longitude'])

Filtramos únicamente aquellos registros que pasaron o no pasaron la prueba

In [ ]:
passed = rest[rest['Results']=='Pass']
failed = rest[rest['Results']=='Fail']

In [ ]:
len(passed), len(failed)

Imagen de la ciudad de chicago:

In [ ]:
chicago = plt.imread('../images/chicago.png')

In [ ]:
boundires = ((rest.Longitude.min(),   rest.Longitude.max(),      
         rest.Latitude.min(), rest.Latitude.max()))

In [ ]:
fig, ax = plt.subplots(1,2, sharex=True, sharey=True, figsize = (20,20))

ax[0].scatter(passed.Longitude, passed.Latitude, zorder=2, alpha=0.1, c='b', s=2)
ax[0].imshow(chicago, zorder=0, extent=boundires, aspect='equal')
ax[0].title.set_text('Results: Pass')

ax[1].scatter(failed.Longitude, failed.Latitude, zorder=2, alpha=0.1, c='r', s=2)
ax[1].imshow(chicago, zorder=0, extent=boundires, aspect='equal')
ax[1].title.set_text('Results: Fail')

plt.show()

---
---